In [1]:
!pip install SQLAlchemy pandas keras pymysql cryptography scikit-learn numpy matplotlib tensorflow

In [2]:
!pip install --upgrade tensorflow keras

  Using cached keras-3.0.4-py3-none-any.whl.metadata (4.8 kB)


In [3]:
# importing libs for ml model
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler

from sqlalchemy import create_engine
from pyspark.sql import SparkSession

2024-01-22 23:24:51.424012: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-22 23:24:51.442951: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-22 23:24:51.487848: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 23:24:51.487894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 23:24:51.527243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [4]:
# connecting with mysql
MYSQL_USER = os.environ.get('MYSQL_USER', 'root')
MYSQL_PASSWORD = os.environ.get('MYSQL_PASSWORD', 'secret')
MYSQL_DATABASE = os.environ.get('MYSQL_DATABASE', 'mydatabase')
MYSQL_DC_HOSTNAME = os.environ.get('MYSQL_DC_HOSTNAME', 'bigdata-mysql')

URL_CONNECT = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_DC_HOSTNAME}:3306/{MYSQL_DATABASE}'

MYSQL_CONNECTION = create_engine(URL_CONNECT)

In [5]:
# --------------------------- Test

In [6]:
# query = "SELECT * FROM stock_data limit 1439833;"
# df = pd.read_sql_query(query, con=MYSQL_CONNECTION)
# df

In [7]:
# -------------------------- End Test

In [8]:
#
# Configuring Spark to execute queries
#

mysql_connector_path = "./mysql-connector-j-8.1.0.jar"
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--jars {mysql_connector_path} --master spark://bigdata-spark:7077 pyspark-shell'

mysql_url = "jdbc:mysql://bigdata-mysql:3306/mydatabase"
mysql_properties = {
    "driver": "com.mysql.cj.jdbc.Driver",
    "user": "user",
    "password": "secret"
}

In [9]:
#
# Spark - Creating session
#
spark = SparkSession.builder.appName("MySQLDataRead").getOrCreate()

24/01/22 23:24:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
# Reading data by MySQL and converting to a spark dataframe
base = spark.read \
    .format("jdbc") \
    .option("url", mysql_url) \
    .option("dbtable", "stock_data") \
    .option("user", mysql_properties["user"]) \
    .option("password", mysql_properties["password"]) \
    .option("driver", mysql_properties["driver"]) \
    .load()

In [11]:
from pyspark.sql.functions import desc

ticker_filtered_df = base.filter(base["Ticker"] == "BBAS3")

# Ordenar por data em ordem decrescente
ticker_filtered_df = ticker_filtered_df.orderBy(desc("Date"))

# Selecionar a primeira linha (mais recente)
data_mais_recente_df = ticker_filtered_df.limit(20)

# Mostrar o DataFrame resultante
data_mais_recente_df.show()


+----------+------------------+-------+------+------------------+------------------+------------------+--------+---------+------------+
|      Date|              Open|Country|Ticker|              High|               Low|             Close|  Volume|Dividends|Stock Splits|
+----------+------------------+-------+------+------------------+------------------+------------------+--------+---------+------------+
|2024-01-12| 54.79999923706055| brazil| BBAS3| 55.20000076293945|54.310001373291016|55.119998931884766| 5063300|      0.0|         0.0|
|2024-01-11|54.400001525878906| brazil| BBAS3|54.849998474121094|54.220001220703125| 54.79999923706055| 6743100|      0.0|         0.0|
|2024-01-10| 54.16999816894531| brazil| BBAS3| 54.66999816894531| 54.16999816894531| 54.43000030517578| 4939300|      0.0|         0.0|
|2024-01-09| 55.09000015258789| brazil| BBAS3|55.150001525878906|54.279998779296875| 54.52000045776367| 7683800|      0.0|         0.0|
|2024-01-08| 55.04999923706055| brazil| BBAS3| 5

In [12]:
# -------------------------- Model

In [13]:
# Assuming that 'base' is already loaded using Spark DataFrame
# You can replace the code below with your Spark DataFrame operations

# Assuming 'Open' column contains the stock prices
base_treinamento = base.select('Open').toPandas().dropna()
# base_treinamento = pd.DataFrame(base.select('Open').collect(), columns=['Open']).dropna()

24/01/22 23:25:05 ERROR TaskSchedulerImpl: Lost executor 1 on 192.168.128.7: Command exited with code 137
24/01/22 23:25:05 WARN TaskSetManager: Lost task 0.0 in stage 1.0 (TID 1) (192.168.128.7 executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Command exited with code 137
24/01/22 23:25:06 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.128.8: Command exited with code 137
24/01/22 23:25:06 WARN TaskSetManager: Lost task 0.1 in stage 1.0 (TID 2) (192.168.128.8 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Command exited with code 137
24/01/22 23:25:10 ERROR TaskSchedulerImpl: Lost executor 2 on 192.168.128.7: Command exited with code 137
24/01/22 23:25:10 WARN TaskSetManager: Lost task 0.2 in stage 1.0 (TID 3) (192.168.128.7 executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Command exited with code 137
24/01/22 23:25:13 ERROR TaskSchedulerImp

Py4JJavaError: An error occurred while calling o46.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 4) (192.168.128.8 executor 3): ExecutorLostFailure (executor 3 exited caused by one of the running tasks) Reason: Command exited with code 137
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2844)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2780)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2779)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2779)
	at org.apache.spar

In [ ]:
# Feature scaling
normalizador = MinMaxScaler(feature_range=(0, 1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [ ]:
# Creating input sequences and labels
previsores = []
preco_real = []

for i in range(90, len(base_treinamento_normalizada)):
    previsores.append(base_treinamento_normalizada[i-90:i, 0])
    preco_real.append(base_treinamento_normalizada[i, 0])

previsores, preco_real = np.array(previsores), np.array(preco_real)
previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))

In [ ]:
# Building the LSTM model
regressor = Sequential()
regressor.add(LSTM(units=100, return_sequences=True, input_shape=(previsores.shape[1], 1)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units=1, activation='linear'))

In [ ]:
regressor.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
regressor.fit(previsores, preco_real, epochs=100, batch_size=32)

In [ ]:
# Preparing test data
base_teste = spark.read \
    .format("jdbc") \
    .option("url", mysql_url) \
    .option("dbtable", "stock_data") \
    .option("user", mysql_properties["user"]) \
    .option("password", mysql_properties["password"]) \
    .option("driver", mysql_properties["driver"]) \
    .load()

preco_real_teste = base_teste.select('Open').toPandas().iloc[:, 0].values

In [ ]:
base_completa = pd.concat((base_treinamento['Open'], preco_real_teste), axis=0)
entradas = base_completa[len(base_completa) - len(preco_real_teste) - 90:].values
entradas = entradas.reshape(-1, 1)
entradas = normalizador.transform(entradas)

X_teste = []

for i in range(90, len(entradas)):
    X_teste.append(entradas[i-90:i, 0])

X_teste = np.array(X_teste)
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))

In [ ]:
# Making predictions
previsoes = regressor.predict(X_teste)
previsoes = normalizador.inverse_transform(previsoes)

# Plotting the results
plt.plot(preco_real_teste, color='red', label='Preço real')
plt.plot(previsoes, color='blue', label='Previsões')
plt.title('Previsão preço das ações')
plt.xlabel('Tempo')
plt.ylabel('Valor Yahoo')
plt.legend()
plt.show()